In [1]:
import numpy as np
import pandas as pd
import regex
import re
from PIL import Image
import torch
from torchvision.transforms import v2
from tqdm import tqdm
import os

from collections import Counter
import sys

In [2]:
# config에서 data 폴더 경로 가져오기
data_dir = "/data/ephemeral/home/data"  # 예: "config/data"

# users.csv 경로 만들기
user_data_path = os.path.join(data_dir, "users.csv")
user_df = pd.read_csv(user_data_path)

book_data_path = os.path.join(data_dir, "books.csv")
book_df = pd.read_csv(book_data_path)

train_rating_data_path = os.path.join(data_dir, "train_ratings.csv")
train_rating_df = pd.read_csv(train_rating_data_path)

## books_df 의 category 피쳐 분석하기

In [5]:


# 이전 질문에서 정의된 문자열을 리스트로 변환하는 함수
def str2list(x: str) -> list:
    '''문자열을 리스트로 변환하는 함수'''
    # 양쪽 대괄호 제거 및 ', ' 기준으로 분할
    return x[1:-1].split(', ')

# --- 고유 카테고리 추출 코드 ---

# 1. 'category' 열의 결측치(NaN)가 아닌 값만 필터링합니다.
#    apply 메서드가 NaN 값에 대해 오류를 일으키지 않도록 방지합니다.
valid_categories = book_df['category'].dropna()

# 2. 모든 카테고리 문자열을 리스트로 변환하고 하나의 리스트로 합칩니다.
#    lambda x: str2list(x)는 각 문자열을 리스트로 변환하고,
#    sum(..., [])는 그 모든 리스트를 하나의 큰 리스트로 결합합니다.
# all_categories = valid_categories.apply(lambda x: str2list(x)).sum(start=[])
all_categories = sum(valid_categories.apply(lambda x: str2list(x)), start=[])
# 3. 합쳐진 리스트에서 고유한(distinct) 카테고리만 추출합니다.
distinct_categories = sorted(list(set(all_categories)))

# 4. 결과를 출력합니다.
print("--- book_df의 고유 카테고리 목록 (Distinct Categories) ---")
print(distinct_categories[:30])


print("--- str2list 역할 파악 ---")
# 이 코드 결과가 없음을 보면 -> str2list 는 그저 문자열을 리스트로 변환하기만 한다.
for category in distinct_categories:
    if len(str2list(category)) >= 2:
        print(str2list(category))

# -----------------------------------------------------------
# 5. 최종 요구사항 분석 및 출력
# -----------------------------------------------------------

# 5-1. 카테고리 빈도 분석 및 TOP 100 출력
category_counts = Counter(all_categories)
df_counts = pd.DataFrame(category_counts.items(), columns=['Category', 'Count'])
df_counts = df_counts.sort_values(by='Count', ascending=False).reset_index(drop=True)

total_category_instances = len(all_categories)
df_counts['Percentage'] = (df_counts['Count'] / total_category_instances) * 100

print(f"장르 종류 갯수: {len(distinct_categories)}개")
print(f"총 books.csv 총 데이터 갯수: {total_category_instances}개")
print("\n[카테고리 등장 빈도 TOP 100 (상위 항목)]")
# Top 100 요청했으나, 데이터가 작으므로 전체 또는 상위 10개만 출력합니다.
top_n = min(100, len(df_counts))
print(df_counts.head(top_n).to_string(index=False, float_format="%.2f"))
print("-" * 50)


# 5-2. 분포 설명 및 이상치 카테고리 출력
# 이상치 정의: 전체 카테고리 중 단 1회만 등장한 카테고리
outlier_categories = df_counts[df_counts['Count'] == 1]

print(f"\n[카테고리 분포 분석]")
print(f"가장 많이 등장한 카테고리: {df_counts.iloc[0]['Category']} ({df_counts.iloc[0]['Count']}회)")
print(f"가장 적게 등장한 카테고리 (이상치): {outlier_categories.shape[0]}개")

if not outlier_categories.empty:
    print("\n[이상치 카테고리 목록 (1회 등장)]")
    # 이상치 목록이 너무 길 경우 10개만 출력
    print(", ".join(outlier_categories['Category'].tolist()[:10]))
print("-" * 50)

# 5-3. 카테고리 이름 글자수 분포 분석
df_distinct = pd.DataFrame({'Category': distinct_categories})
# 카테고리 이름 길이 계산 (공백 포함)
df_distinct['Length'] = df_distinct['Category'].apply(len)

length_stats = df_distinct['Length'].describe()
median_length = length_stats['50%']
std_dev = length_stats['std']

# 이상적으로 길거나 짧은 카테고리 정의 (중앙값 기준 ± 2*표준편차를 벗어나는 경우)
# 예시 데이터가 작으므로, 임의의 기준으로 설정: 짧음(5 미만), 김(15 초과)
SHORT_THRESHOLD = 5
LONG_THRESHOLD = 15

short_categories = df_distinct[df_distinct['Length'] < SHORT_THRESHOLD]
long_categories = df_distinct[df_distinct['Length'] > LONG_THRESHOLD]

print(f"\n[카테고리 이름 글자수 분포 분석]")
print(f"전체 고유 카테고리 수: {len(distinct_categories)}개")
print(f"카테고리 이름 길이 통계:\n{length_stats.to_string()}")

print(f"\n- 글자수가 {SHORT_THRESHOLD} 미만인 짧은 카테고리: {short_categories.shape[0]}개")
if not short_categories.empty:
    print(f"  목록 ({short_categories.shape[0]}개): {', '.join(short_categories['Category'].tolist())}")

print(f"- 글자수가 {LONG_THRESHOLD} 초과인 긴 카테고리: {long_categories.shape[0]}개")
if not long_categories.empty:
    print(f"  목록 ({long_categories.shape[0]}개): {', '.join(long_categories['Category'].tolist())}")
print("-" * 50)


--- book_df의 고유 카테고리 목록 (Distinct Categories) ---
['"ABC\'s"', '"Aesop\'s fables"', '"Almanacs', '"Alzheimer\'s disease"', '"April Fools\' Day"', '"Arthur Miller\'s The Crucible"', '"Artists\' books"', '"Artists\' contracts"', '"Artists\' models"', '"Artists\' representatives"', '"Artists\' spouses"', '"Author\'s spouses"', '"Authors\' spouses"', '"Blake\'s 7 (Television program)"', '"Bug\'s life (Motion picture)"', '"Chefs d\'orchestre - États-Unis - Biographies"', '"Children\'s accidents"', '"Children\'s art"', '"Children\'s atlases"', '"Children\'s audiobooks"', '"Children\'s costumes"', '"Children\'s detective and mystery stories"', '"Children\'s encyclopedias and dictionaries"', '"Children\'s fantasy fiction"', '"Children\'s films"', '"Children\'s literature', '"Children\'s literature"', '"Children\'s literature."', '"Children\'s paraphernalia"', '"Children\'s parties"']
--- str2list 역할 파악 ---
장르 종류 갯수: 4449개
총 books.csv 총 데이터 갯수: 83097개

[카테고리 등장 빈도 TOP 100 (상위 항목)]
             

### category 피쳐 -> bert 이용 임베딩 -> 클러스터링(n 개의 범주로) 

In [7]:
# -------------------------------
# 1. 필수 라이브러리 설치
# -------------------------------
# !pip install sentence-transformers scikit-learn tqdm

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from tqdm import tqdm

# -------------------------------
# 2. 카테고리 전처리
# -------------------------------
def preprocess_category(df, col='category', min_len=3):
    """
    - 결측치 처리
    - 소문자화 및 공백 제거
    - 글자수 min_len 미만은 'other'로 치환
    """
    df[col] = df[col].fillna('Unknown')
    df[col] = df[col].str.strip().str.lower()
    df.loc[df[col].str.len() < min_len, col] = 'other'
    return df

# -------------------------------
# 3. 텍스트 augmentation
# -------------------------------
def augment_category_text(df, col='category'):
    """
    BERT 임베딩 안정화를 위해 단일 단어 카테고리를 문장 형태로 변환
    """
    df['category_text'] = df[col].apply(lambda x: f"Category: {x}")
    return df

# -------------------------------
# 4. Sentence-BERT 임베딩
# -------------------------------
def embed_categories(df, text_col='category_text', model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    categories = df[text_col].tolist()
    embeddings = model.encode(categories, batch_size=64, show_progress_bar=True)
    return np.array(embeddings)

# -------------------------------
# 5. KMeans 클러스터링
# -------------------------------
def cluster_categories(embeddings, n_clusters=200, random_state=42):
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
    cluster_labels = kmeans.fit_predict(embeddings)
    return cluster_labels

# -------------------------------
# 6. 적용 예시
# -------------------------------
# book_df 예시
# book_df = pd.read_csv('books.csv')

# 1) 전처리
book_df = preprocess_category(book_df, col='category', min_len=3)

# 2) 텍스트 augmentation
book_df = augment_category_text(book_df, col='category')

# 3) 임베딩
category_embeddings = embed_categories(book_df, text_col='category_text', model_name='all-MiniLM-L6-v2')

# 4) KMeans 클러스터링
n_clusters = 200  # 필요시 조절
book_df['category_cluster'] = cluster_categories(category_embeddings, n_clusters=n_clusters)

# -------------------------------
# 7. 결과 확인
# -------------------------------
print(book_df[['category', 'category_cluster']].head(10))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2338 [00:00<?, ?it/s]

        category  category_cluster
0  ['actresses']               102
1  ['1940-1949']               143
2    ['medical']                39
3    ['fiction']                 0
4    ['history']                 4
5        unknown                 1
6    ['fiction']                 0
7    ['fiction']                 0
8        unknown                 1
9        unknown                 1


In [10]:
# -------------------------------
# 8. 클러스터별 카테고리 확인
# -------------------------------

# cluster별로 대표 카테고리 확인
cluster_summary = book_df.groupby('category_cluster')['category'].unique().reset_index()

# 보기 편하게 일부만 출력
pd.set_option('display.max_rows', 50)  # 필요시 조절
for idx, row in cluster_summary.iterrows():
    print(f"Cluster {row['category_cluster']}: {row['category']}")


Cluster 0: ["['fiction']" "['humorous fiction']" "['autobiographical fiction']"
 "['authors fiction']" "['fiction, general']"]
Cluster 1: ['unknown']
Cluster 2: ["['juvenile fiction']" "['young adult fiction']"
 "['juvenile science fiction']"
 "['best books for young adults (fiction)']"
 "['juvenile fiction- nasty people, pranks, monkeys']"
 "['brothers and sisters juvenile fiction']"]
Cluster 3: ["['control (psychology)']" "['psychology']"
 "['assertiveness (psychology)']" "['child psychologists']"
 "['attitude (psychology)']" "['developmental psychology']"
 "['identity (psychology)']" "['androgyny (psychology)']"
 "['self-actualization (psychology)']" "['change (psychology)']"
 "['child psychology']" "['human experimentation in psychology']"
 "['adjustment (psychology)']" "['psychoanalysis']"
 "['chang (psychology)']" "['choice (psychology)']" "['sex (psychology)']"
 "['psychology, applied']" "['cognitive psychology']"
 "['gestalt psychology']" "['course - aspect psychologique']"
 "[